In [344]:
### Grading script code 
### You don't need to read this, proceed to the next cell
import sys
import functools
ipython = get_ipython()

def set_traceback(val):
    method_name = "showtraceback"
    setattr(
        ipython,
        method_name,
        functools.partial(
            getattr(ipython, method_name),
            exception_only=(not val)
        )
    )

class AnswerError(Exception):
  def __init__(self, message):
    pass

def exec_test(f, question):
    try:
        f()
        print(question + " Pass")
    except:
        set_traceback(False) # do not remove
        raise AnswerError(question + " Fail")

# Week 12 Problem Set

## Homework

**HW1.** *Comments:* Write a state machine whose inputs are the characters of a string. The string contains the code for a computer program. The output of the state machine are either:
- the input character if it is part of a comment, or
- `None`, otherwise.

Comment starts with a `#` character and continue to the end of the current line. If you want to create a string that contains a new line character, you can use `\n`.

For example,
```
inpstr = "def func(x): # comment\n    return 1"
m = CommentsSM()
print(m.transduce(inpstr))
```

The expected output is:
```
[None, None, None, None, None, None, None, None, None, None, None, None, None, "#", " ", "c", "o", "m", "m", "e", "n", "t", None, None, None, None, None, None, None, None, None, None, None, None, None]
```

You should start by drawing a state transition diagram indicating the states and what inputs cause transition to which other states. Use the test case above to determine if your state transition diagram is correct. You should begin writing your program only when you are confident that your diagram is correct. 

In [345]:
# Copy over the implementation of StateMachine from Cohort
from abc import ABC, abstractmethod

class StateMachine(ABC):

    @property
    @abstractmethod
    def start_state(self):
        pass

    def start(self):
        self.state = self.start_state

    def step(self, inp):
        ns, o = self.get_next_values(self.state, inp)
        self.state = ns
        return o 

    def transduce(self, inp_list):
        self.start()
        return [self.step(inp) for inp in inp_list if not self.is_done()] 

    @abstractmethod
    def get_next_values(self, state, inp):
        pass

    def done(self, state):
        return False

    def is_done(self):
        return self.done(self.state)

In [346]:
class CommentsSM(StateMachine):

    start_state = False

    # each inp is a character
    def get_next_values(self, state, inp):
        
        if state:
            if inp != '\n':
                next_state = state
                output = inp
            else:
                next_state = False
                output = None
        
        else:
            if inp == '#':
                next_state = True
                output = inp
            else:
                next_state = False
                output = None
        
        return next_state, output

In [347]:
inpstr = "def func(x): # comment\n    return 1"
m = CommentsSM()
out = m.transduce(inpstr)
assert out == [None, None, None, None, None, None, None, None, None, None, None, None, None, "#", " ", "c", "o", "m", "m", "e", "n", "t", None, None, None, None, None, None, None, None, None, None, None, None, None]

In [348]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** *First Word:* Write a state machine whose inputs are the characters of a string and which outputs either:
- the input character if it is part of the first word on a line, or
- `None`, otherwise

For the purposes here, a word is any sequence of consecutive characters that does not contain spaces or end-of-line characters. In this problem, comments have no special status. This means that if the line begins with `# `, then the first word is `#`. 

For example, 
```
inpstr = "def func(x): # comment\n    return 1"
m = FirstWordSM()
print(m.transduce( inpstr))
```

The expected output is:
```
["d", "e", "f", None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, "r", "e", "t", "u", "r", "n", None, None]
```

In [349]:
class FirstWordSM(StateMachine):
    start_state = 1
  
    def get_next_values(self, state, inp):
        # 0 to 1 state is whether \n has been encountered or not
        # 1 to 2 state is whether first letter has been encountered

        if state == 1:
            if not (inp == ' ' or inp == '\n'):
                next_state = 2
                output = inp
            else:
                next_state = state
                output = None
        elif state == 2:
            if inp == ' ':
                next_state = 0
                output = None
            elif inp == '\n':
                next_state = 1
                output = None
            else:
                next_state = state
                output = inp
        else:
            if inp == '\n':
                next_state = 1
                output = None
            else:
                next_state = state
                output = None

        return next_state, output

In [350]:
inpstr = "def func(x): # comment\n    return 1"
m = FirstWordSM()
out = m.transduce(inpstr)
assert out == ["d", "e", "f", None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, "r", "e", "t", "u", "r", "n", None, None]


In [351]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** *Robot:* Write a State Machine class that represent a robot. The dimension of the world and the robot initial position should be specified during the class instantiation. The robot can take in the following input:
- "left"
- "right"
- "up"
- "down"

The initial position of the robot is specified during the object instantiation and the input should modify the position of the robot. The robot position must not change if it exceed the boundary. At each step, the robot should output the updated position. 

In [352]:
class Position:
  
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
  
    def __str__(self):
        return f"({self.x:}, {self.y:})"

class Dimension:

    def __init__(self, width=0, height=0):
        self.width = width
        self.height = height
  
    def __str__(self):
        return f"width: {self.width:}, height: {self.height:}"

In [353]:
class RobotSM(StateMachine):

    def __init__(self, init_pos, dimension):
        self.world_dim = dimension
        self.start_state = init_pos

    @property
    def start_state(self):
        return self._start_state
    
    @start_state.setter
    def start_state(self, pos):
        if pos.x <= self.world_dim.width and pos.x >=0 and pos.y <= self.world_dim.height and pos.y >= 0:
            self._start_state = pos

    def get_next_values(self, state, inp):
        
        x_pos = state.x
        y_pos = state.y
        if inp == 'right' and x_pos + 1 <= self.world_dim.width:
            x_pos += 1
        elif inp == 'left' and x_pos - 1 >= 0:
            x_pos -= 1
        elif inp == 'up' and y_pos + 1 <= self.world_dim.height:
            y_pos +=1
        elif inp == 'down' and y_pos - 1 >= 0:
            y_pos -= 1

        new_state = Position(x_pos, y_pos)
        return new_state, new_state


In [354]:
robot = RobotSM(Position(0, 0), Dimension(5, 5))
robot.start()
robot.transduce(["right", "right", "up", "up", "up", "left", "down"])
pos = robot.state
assert pos.x == 1 and pos.y == 2

In [355]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** *Search SM:* Write a function `sm_search` that takes in the following arguments:
- `sm_to_search`: is the State Machine instance to search. This argument is of the type `MapSM` as defined in CS4. You should use the `get_next_values()` of this State Machine instance to explore the next state in your search.
- `initial_state`: is the start state of the search. If it is not provided, it should be assigned to the `start_state` of sm_search. 
- `goal_test`: is a function that returns `True` if the argument is the end state of the search. If it is not provided, it should be eassigned to the `done` function of the state machine.

This function performs a **breadth-first-search** algorithm to explore the next states. 

The output is a `list` of `Step` instances from the `init_state` to the end state which is determined by the `goal_test` function.

This problem requires you to complete the following:
- `Queue` class from Problem Set 4 HW2.
- `MapSM` class in CS4.
- `SearchNode` and `Step` classes in CS5.

In [356]:
# Copy over the implementations of Queue from PS4 HW2
class Queue:
    def __init__(self):
        self.ls = []

    def enqueue(self, item):
        self.ls.append(item)
    
    def dequeue(self):
        if not self.is_empty:
            return self.ls.pop(0)

    @property
    def is_empty(self):
        return self.ls == []

In [357]:
from abc import abstractmethod

class StateSpaceSearch(StateMachine):
    @property
    @abstractmethod
    def statemap(self):
        pass

    @property
    @abstractmethod
    def legal_inputs(self):
        pass


In [358]:
# Copy over the implementation of MapSM from Cohort
class MapSM(StateSpaceSearch):
    
    def __init__(self, start):
        self.start_state = start
        
    @property
    def start_state(self):
        return self._start_state

    @start_state.setter
    def start_state(self, node):
        if node in self.statemap.keys():
            self._start_state = node
        else:
            self._start_state = 'S'
    @property
    def statemap(self):
        statemap = {"S": ["A", "B"],
                    "A": ["S", "C", "D"],
                    "B": ["S", "D", "E"],
                    "C": ["A", "F"],
                    "D": ["A", "B", "F", "H"],
                    "E": ["B", "H"],
                    "F": ["C", "D", "G"],
                    "H": ["D", "E", "G"],
                    "G": ["F", "H"]}
        return statemap

    @property
    def legal_inputs(self):
        max = -1
        for inps in self.statemap.values():
            curr_inps_len = len(inps)
            if curr_inps_len > max:
                max = curr_inps_len
        
        return set(range(max))
    def get_next_values(self, state, inp):
        neighbours = self.statemap.get(state)
        next_state = state if inp >= len(neighbours) else neighbours[inp]
        return next_state, next_state

In [359]:
# Copy over the implementations of Step and SearchNode from Cohort
class Step:
    def __init__(self, action, state):
        self.action = action
        self.state = state
    
    def __eq__(self, other):
        return self.action == other.action and self.state == other.state
  
    def __str__(self):
        return f"action: {self.action:}, state: {self.state:}"

class SearchNode:
    def __init__(self, action, state, parent=None):
        self.state = state
        self.action = action
        self.parent = parent
  
    def path(self):
        if self.parent == None:
            return [Step(self.action, self.state)]
        else:
            return self.parent.path() + [Step(self.action, self.state)] 


    def in_path(self, state):
        if self.state == state:
            return True
        elif self.parent == None:
            return False
        else:
            return self.parent.in_path(state)
  
    def __eq__(self, other):
        if self is None and other is None:
            return True
        elif self is None:
            return False
        elif other is None:
            return False
        else:
            return self.state == other.state and self.parent == other.parent and \
                   self.action == other.action

**HW4.** *Search SM:* Write a function `sm_search` that takes in the following arguments:
- `sm_to_search`: is the State Machine instance to search. This argument is of the type `MapSM` as defined in CS4. You should use the `get_next_values()` of this State Machine instance to explore the next state in your search.
- `initial_state`: is the start state of the search. If it is not provided, it should be assigned to the `start_state` of sm_search. 
- `goal_test`: is a function that returns `True` if the argument is the end state of the search. If it is not provided, it should be eassigned to the `done` function of the state machine.

This function performs a **breadth-first-search** algorithm to explore the next states. 

The output is a `list` of `Step` instances from the `init_state` to the end state which is determined by the `goal_test` function.

This problem requires you to complete the following:
- `Queue` class from Problem Set 4 HW2.
- `MapSM` class in CS4.
- `SearchNode` and `Step` classes in CS5.

In [360]:
def sm_search(sm_to_search, initial_state=None, goal_test=None):
    # check if initial_state is provided
    # if it is, use it
    # otherwise, get the start state of sm_to_search
    if initial_state == None:
        # replace None to take the start state of sm_to_search
        init_state = sm_to_search.start_state
        ###
        ### YOUR CODE HERE
        ###
    else:
        init_state = initial_state
  
    # check if goal_test is provided
    # if it is, use it
    # otherwise, use the done method as the goal function
    # taken from sm_to_search
    if goal_test == None:
        goal_func = sm_to_search.done
    else:
        goal_func = goal_test
  
    # create a Queue instance to store the node to explore
    # replace the None below
    agenda = Queue()
    ###
    ### YOUR CODE HERE
    ###
  
    # if the initial state is the goal state, 
    # then we are done and exit
    if goal_func(init_state):
        return [Step(None, init_state)]
  
    # otherwise, add the current node into the agenda 
    agenda.enqueue(SearchNode(None, init_state, None))
    
    # explore as long as the Queue is not empty
    while not agenda.is_empty:
    
        # replace None to take out the parent from the Queue
        parent = agenda.dequeue()
        ###
        ### YOUR CODE HERE
        ###
        
        # create a list to keep track which child state have been explored
        new_child_state = [] 
        
        # get all the legal input values
        actions = sm_to_search.legal_inputs

        #iterate over all legal inputs
        for a in actions:
            # get the next possible state using the current action
            # call get_next_values to get the next state
            # replace the None below
            new_s, o = sm_to_search.get_next_values(parent.state, a)
            if new_s == parent.state:
                continue
            ###
            ### YOUR CODE HERE
            ###
            
            # create a new search node from the new_s
            # replace the None below
            new_n = SearchNode(action=a, state=new_s, parent=parent)
            ###
            ### YOUR CODE HERE
            ###
            
            # if the new state is the goal state, then we exit and return the path
            if goal_func(new_s):
                return new_n.path()
            
            # if the new state is already in the list of new child state, ignore it
            elif new_s in new_child_state:
                pass
            
            # if the new state is in the path of the current node, ignore it
            elif parent.in_path(new_s):
                pass
            
            # otherwise, add the new state into the list
            # and the new node into the Queue
            else:
                # step 1. add the new state into the new_child_state
                ###
                ### YOUR CODE HERE
                ###
                new_child_state.append(new_s)
                
                # step 2. add the new node into the Queue
                ###
                ### YOUR CODE HERE
                ###
                agenda.enqueue(new_n)
    return None

In [361]:
mapSM = MapSM("S")
ans = sm_search(mapSM , "S" , lambda s: s=="H" )
steps = [(step.action, step.state) for step in ans]
assert steps == [(None, "S"), (0, "A"), (2, "D"), (3, "H")]
for step in ans:
    print(step)

action: None, state: S
action: 0, state: A
action: 2, state: D
action: 3, state: H


In [362]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
